In [ ]:
import os
import pickle
import time
import numpy as np
import json
import matplotlib.pyplot as plt
from types import SimpleNamespace
from collections import defaultdict
from numba import jit

# utils imports
from power_planner.data_reader import DataReader
from power_planner import graphs
from power_planner.plotting import plot_path_costs, plot_pipeline_paths, plot_path, plot_k_sp
from power_planner.utils.utils import get_distance_surface, time_test_csv, compute_pylon_dists
from power_planner.utils.utils_ksp import KspUtils

In [ ]:
PATH_FILES = "../data"

# DEFINE CONFIGURATION
ID = "w_ksp_5"  # str(round(time.time() / 60))[-5:]

OUT_PATH = "outputs/path_" + ID
SCALE_PARAM = 2  # args.scale
# normal graph pipeline
# PIPELINE = [(2, 50), (1, 0)]  # [(1, 0)]  # [(4, 80), (2, 50), (1, 0)]  #
# random graph pipeline
PIPELINE = [(1, 0)]  # [(0.9, 40), (0, 0)]

GRAPH_TYPE = graphs.ImplicitKSP
# LineGraph, WeightedGraph, RandomWeightedGraph, RandomLineGraph, PowerBF
# TwoPowerBF, WeightedKSP
print("graph type:", GRAPH_TYPE)
# summarize: mean/max/min, remove: all/surrounding, sample: simple/watershed
NOTES = "None"  # "mean-all-simple"

IOPATH = os.path.join(PATH_FILES, "belgium_dump_" + str(SCALE_PARAM) + ".dat")

with open("../config.json", "r") as infile:
    cfg_dict = json.load(infile)  # Config(SCALE_PARAM)
    cfg = SimpleNamespace(**cfg_dict)
    cfg.PYLON_DIST_MIN, cfg.PYLON_DIST_MAX = compute_pylon_dists(
        cfg.PYLON_DIST_MIN, cfg.PYLON_DIST_MAX, cfg.RASTER, SCALE_PARAM
    )

In [ ]:
# READ DATA
with open(IOPATH, "rb") as infile:
    data = pickle.load(infile)
    (instance, instance_corr, start_inds, dest_inds) = data.data

In [ ]:
tic1 = time.time()
graph = GRAPH_TYPE(
    instance, instance_corr, graphtool=cfg.GTNX, verbose=cfg.VERBOSE
)

graph.set_shift(
    cfg.PYLON_DIST_MIN,
    cfg.PYLON_DIST_MAX,
    dest_inds - start_inds,
    cfg.MAX_ANGLE,
    max_angle_lg=cfg.MAX_ANGLE_LG
)
corridor = np.ones(instance_corr.shape) * 0.5  # start with all

graph.set_corridor(corridor, start_inds, dest_inds, factor_or_n_edges=1)

graph.set_edge_costs(
    data.layer_classes, data.class_weights, angle_weight=cfg.ANGLE_WEIGHT
)
# add vertices
graph.add_nodes()

# START PIPELINE
tic = time.time()
print("1) set cost rest")
graph.add_edges()
print("2) added edges", graph.n_edges)
print("number of vertices:", graph.n_nodes)

# weighted sum of all costs
graph.sum_costs()
source_v, target_v = graph.add_start_and_dest(start_inds, dest_inds)
print("3) summed cost, get source and dest")
# get actual best path
path, path_costs, cost_sum = graph.get_shortest_path(source_v, target_v)
print("4) shortest path")
print(time.time()-tic1)

In [ ]:
graph.get_shortest_path_tree(source_v, target_v)

### eucl distance impl ksp

In [ ]:
ksp = graph.k_shortest_paths(source_v, target_v, 5)

### eucl max dist greedy

In [ ]:
def ksp_eucl_max(self, source, dest, k, overlap=0.5):
    min_node_dists, v_shortest, min_shift_dists = compute_min_node_dists(self)  
    best_paths = [self.best_path]
    tup_path = [np.array(p) for p in self.best_path]
    # sp_set = set(tuple_path)
    sorted_dists = min_node_dists.flatten()[v_shortest]
    _, arr_len  = min_node_dists.shape

    expanded = 0
    for j in range(len(v_shortest)):
        if sorted_dists[j]==sorted_dists[j-1]:
            # we always check a path only if it is the x-th appearance
            # print(counter)
            continue
         
        # counter large enough --> expand
        (x2, x3) = v_shortest[j]//arr_len, v_shortest[j]%arr_len
        
        # compute eucledian distances
        eucl_dist = [np.linalg.norm(np.array([x2,x3]) - tup) for tup in tup_path]
        if np.min(eucl_dist)>overlap:
            expanded += 1
            x1 = min_shift_dists[x2,x3]
            if self.dists_ba[x1, x2, x3] == 0:
                # print("inc edge to dest")
                # = 0 for inc edges of dest_inds (init of dists_ba)
                continue
            vertices_path = self._combined_paths(
                source, dest, x1, [x2, x3]
            )
            best_paths.append(vertices_path)
            tup_path.extend(list(vertices_path))

            if len(best_paths) >= k:
                print(j, "expanded", expanded)
                break

    return [self.transform_path(path) for path in best_paths]

In [ ]:
tic_overall = time.time()
ksp = ksp_eucl_max(graph, start_inds, dest_inds, 5, overlap = 30)
print("time", time.time()-tic_overall)

In [ ]:
plot_k_sp(ksp, graph.instance)

### counter

In [ ]:
def ksp_w_counter(self, source, dest, k, overlap=0.5):
    min_node_dists, v_shortest, min_shift_dists = compute_min_node_dists(self)  
    best_paths = [self.best_path]
    tuple_path = [tuple(p) for p in self.best_path]
    sp_set = set(tuple_path)
    sorted_dists = min_node_dists.flatten()[v_shortest]
    _, arr_len  = min_node_dists.shape
    COUNT_THRESH = 4
    counter = 0
    start = 0
    expanded = 0
    for j in range(len(v_shortest)):
        if sorted_dists[j]==start:
            # we always check a path only if it is the x-th appearance
            counter += 1
            # print(counter)
            continue
            
        if counter < COUNT_THRESH:
            counter = 1
            start = sorted_dists[j]
            continue
        # reset counter
        counter = 0
        expanded += 1
        # counter large enough --> expand
        (x2, x3) = v_shortest[j]//arr_len, v_shortest[j]%arr_len
        x1 = min_shift_dists[x2,x3]
        if self.dists_ba[x1, x2, x3] == 0:
            # print("inc edge to dest")
            # = 0 for inc edges of dest_inds (init of dists_ba)
            continue
        vertices_path = self._combined_paths(
            source, dest, x1, [x2, x3]
        )
        # compute similarity with previous paths
        # TODO: similarities
        already = np.array([tuple(u) in sp_set for u in vertices_path])
        # print(already)
        if np.sum(already) < len(already) * overlap:
            best_paths.append(vertices_path)
            tup_path = [tuple(p) for p in vertices_path]
            sp_set.update(tup_path)
            # _, _, cost = self.transform_path(vertices_path)
            # print("found new path with cost", cost)
            # print("sorted dist:", sorted_dists[j])
        if len(best_paths) >= k:
            print(j, "expanded", expanded)
            break
    
        start = sorted_dists[j]
    return [self.transform_path(path) for path in best_paths]

## Implicit KSP with vertices

In [ ]:
def k_shortest_paths(self, source, dest, k, overlap=0.5):
    tic = time.time()

    best_paths = [self.best_path]
    tuple_path = [tuple(p) for p in self.best_path]
    sp_set = set(tuple_path)
    # sum both dists_ab and dists_ba, subtract inst because counted twice
    summed_dists = (self.dists + self.dists_ba - self.instance)
    # mins along outgoing edges
    min_node_dists = np.min(summed_dists, axis=0)
    self.min_node_dists = min_node_dists
    min_shift_dists = np.argmin(summed_dists, axis=0)
    # argsort
    v_shortest = np.argsort(min_node_dists.flatten())
    _, arr_len  = min_node_dists.shape
    c1=0
    c2=0
    t1, t2 = [], []
    times_expand, times_compare = [], []
    expanded = 0
    # e_shortest = np.argsort(min_node_dists.flatten())
    # sorted dists:
    sorted_dists = min_node_dists.flatten()[v_shortest]
    # for (i,j) in self.best_path:
     #   print(min_node_dists[i,j])
    # print(sp_set)
    # iterate over edges from least to most costly
    for j in range(len(v_shortest)):
        if sorted_dists[j] == sorted_dists[j - 1]:
            # already checked
            c1+=1
            continue
        (x2, x3) = v_shortest[j]//arr_len, v_shortest[j]%arr_len
        x1 = min_shift_dists[x2,x3]
        if self.dists_ba[x1, x2, x3] == 0:
            # print("inc edge to dest")
            # = 0 for inc edges of dest_inds (init of dists_ba)
            continue
        tic1 = time.time()
        vertices_path = self._combined_paths(
            source, dest, x1, [x2, x3]
        )
        times_expand.append(time.time()-tic1)
        expanded += 1
        # compute similarity with previous paths
        # TODO: similarities
        tic2 = time.time()
        already = np.array([tuple(u) in sp_set for u in vertices_path])
        # if similarity < threshold, add
        if np.sum(already) < len(already) * overlap:
            best_paths.append(vertices_path)
            tup_path = [tuple(p) for p in vertices_path]
            sp_set.update(tup_path)
            # _, _, cost = self.transform_path(vertices_path)
            # print("found new path with cost", cost)
            # print("sorted dist:", sorted_dists[j])
        times_compare.append(time.time()-tic2)
        if len(best_paths) >= k:
            print(j, "expanded", expanded)
            break
        # if j%1000==0:
        #     print(np.mean(times_expand), np.mean(times_compare))
        #     print(j, "expand", expanded, "left out", c2)

    self.time_logs["ksp"] = round(time.time() - tic, 3)
    return [self.transform_path(path) for path in best_paths]

In [ ]:
plt.figure(figsize=(20,10))
plt.imshow(graph.min_node_dists)
plt.scatter(dest_inds[1], dest_inds[0])
plt.show()
# problem: incoming edges to dest ind --> cannot be zero

In [ ]:
tic = time.time()
ksp = k_shortest_paths(graph, start_inds, dest_inds, 5)
print(time.time()-tic)

#### Validate results

In [ ]:
plt.figure(figsize= (20,10))
plt.imshow(instance_corr)
for p in paths:
    p = np.array(p)
    plt.scatter(p[:,1], p[:,0])
    for (i,j) in p:
        if instance_corr[i,j]==0:
            print(i,j)

In [ ]:
with open("../../outputs/testksp_ksp.json", "r") as outfile:
    ksp = json.load(outfile)

In [ ]:
paths = [k[0] for k in ksp]

In [ ]:
p1 = np.array(paths[0])

### Baseline

In [ ]:
def short_eval(ksp):
    # compute sum of all costs
    return np.sum([k[2] for k in ksp])

In [ ]:
ksp = graph.k_shortest_paths(source_v, target_v,10, overlap=0.2, mode="IoU")

In [ ]:
ksp_evaluate(ksp)

In [ ]:
plot_k_sp(ksp, graph.instance)

## Optimize KSP

In [ ]:
best_path_cells, _, best_cost = graph.transform_path(graph.best_path)
best_path = graph.best_path

In [ ]:
best_cost

In [ ]:
# get shortest paths
vertices = np.unique(graph.pos2node)[1:]
v_costs = [graph.instance[(ind // graph.y_len, ind % graph.y_len)] for ind in vertices]
v_dists = [graph.dist_map_ab[v] + graph.dist_map_ba[v] for i, v in enumerate(vertices)]
v_shortest = np.argsort(v_dists)
sorted_dists = np.array(v_dists)[v_shortest]

In [ ]:
graph.instance[tuple(dest_inds)]/2, graph.instance[start_inds[0], start_inds[1]]/2

In [ ]:
sorted_dists[0], v_shortest[0]

In [ ]:
7.4533313605426 + 0.12793022374845409 + 0.07995270935960591

In [ ]:
# proof that they are not the same!!
opt_path = compute_sp(graph, vertices[v_shortest[0]], source_v, target_v)
for b in range(len(best_path)):
    if (opt_path[b]!=best_path[b]):
        print(opt_path[b],best_path[b])

In [ ]:
opt_path = simple_transform()

In [ ]:
def compute_sp(self, v, source, dest):
    path_ac = self.get_sp_from_preds(self.pred_map_ab, v, source)
    path_cb = self.get_sp_from_preds(self.pred_map_ba, v, dest)
    # times_getpath.append(time.time() - tic1)
    path_ac.reverse()
    # concatenate - leave 1 away because otherwise twice
    vertices_path = path_ac + path_cb[1:]
    return vertices_path

In [ ]:
# params
sim_mode = "IoU"
k = 10
overlap = 1

### get only ksp:

In [ ]:
overall_set = set(best_path)
ksp_paths = [best_path]
for j, v_ind in enumerate(v_shortest):
    v = vertices[v_ind]
    if v not in overall_set:
        vertices_path = compute_sp(graph, v, source_v, target_v)
        overall_set.update(vertices_path)
        ksp_paths.append(vertices_path)
        # p, _, cost = graph.transform_path(vertices_path)
        # print(cost)
    if len(ksp_paths)>=k:
        break

In [ ]:
ksp_ksp = [graph.transform_path(p) for p in ksp_paths]

In [ ]:
cost_sum = short_eval(ksp_ksp)

In [ ]:
c_approx = 1.01
max_costs = c_approx * cost_sum

In [ ]:
max_costs/10

In [ ]:
c_approx = 1.025
max_costs = c_approx*best_cost
print(max_costs)

In [ ]:
def eucl_dist(p1, p2):
    dists = []
    for p in range(min([len(p1), len(p2)])):
        dists.append(np.linalg.norm(p1[p]-p2[p]))
    return np.mean(dists)
    
def similarity(s1, s2, mode = "IoU"):
    path_inter = len(s1.intersection(s2))
    if mode == "IoU":
        return path_inter / len(s1.union(s2))
    else:
        raise NotImplementedError("mode must be iou")
        
@jit(nopython=True)
def compute_eucl(path1,path2, mode="mean"):
    min_dists_out = np.zeros(len(path1))
    for p1 in range(len(path1)):
        min_dists = np.zeros(len(path2))
        for p2 in range(len(path2)):
            min_dists[p2] = np.linalg.norm(path1[p1]-path2[p2])
        min_dists_out[p1] = np.min(min_dists)
    if mode=="mean":
        return np.mean(min_dists_out)
    elif mode=="max":
        return np.max(min_dists_out)
            
        
def path_distance(p1,p2,mode="jaccard"):
    """
    modes: Jaccard: jaccard distance (1-IoU)
            eucl_mean: from all min eucl distances, take mean
            eucl_max: from all min eucl distances, take max
    """
    if mode=="jaccard":
        s1 = set([tuple(p) for p in p1])
        s2 = set([tuple(p) for p in p2])
        # s1,s2 = (set(list(p1)),set(list(p2)))
        return 1 - len(s1.intersection(s2))/len(s1.union(s2))
    elif mode.startswith("euc"):
        p1 = np.array(p1).astype("float")
        p2 = np.array(p2).astype("float")
        eucl_mode = mode.split("_")[1]
        return max([compute_eucl(p1,p2,mode=eucl_mode), compute_eucl(p2,p1,mode=eucl_mode)])

In [ ]:
path_distance()

In [ ]:
start = 0
counter = 20
dist_arr = []
collected_paths = []
for c in range(len(v_shortest)):
    new = sorted_dists[c]
    # check whether exactly the same 
    if np.isclose(new, start):
        counter +=1
        continue
        
    # counter: tells me how many new nodes this path has in contrast to the one before --> skip if not very different  
    if counter<10:
        counter = 1
        start = new
        continue
        
    # elidgible: compute path
    # print(start, counter)
    vertices_path = compute_sp(graph, vertices[v_shortest[c]], source_v, target_v)
    if vertices_path[0]!=source_v or vertices_path[-1]!= target_v:
        print(vertices[v_shortest[c]])
        print(vertices_path)
        raise RuntimeError("source or target not contained")
    collected_paths.append(vertices_path)
    
    # renew the current mindist
    start = new
    counter = 1
    
    # stop to collect paths if costs too high
    if start>max_costs:
        break

In [ ]:
len(collected_paths)

In [ ]:
collected_coords = [simple_transform(p) for p in collected_paths]

In [ ]:
dists = np.zeros((len(collected_coords), len(collected_coords)))
for i in range(len(collected_coords)):
    for j in range(i, len(collected_coords)):
        dists[i,j] = path_distance(collected_coords[i], collected_coords[j],mode="jaccard")
        dists[j,i] = dists[i,j]

In [ ]:
plt.imshow(dists)
plt.colorbar()
plt.show()

In [ ]:
nr_paths = len(collected_sets)
d1 = np.argmax(dists)// nr_paths
d2 = np.argmax(dists)% nr_paths

In [ ]:
d1,d2

In [ ]:
collected_paths[d1][0], collected_paths[d1][-1], collected_paths[d2][0], collected_paths[d2][-1]

In [ ]:
nr_paths

In [ ]:
def plot_path(path):
    coords, _, _ = graph.transform_path(path)
    coords = np.array(coords)
    plt.plot(coords[:,0], coords[:,1])

In [ ]:
plot_path(collected_paths[d1])
plot_path(collected_paths[d2])
div_ksp = [d1,d2]
for _ in range(8):
    min_dists = []
    for i in range(nr_paths):
        min_dists.append(np.min([dists[i,div_ksp]]))
    d3 = np.argmax(min_dists)
    # print(collected_paths[d3][0], collected_paths[d3][-1])
    # print(d3)
    div_ksp.append(d3)
    plot_path(collected_paths[d3])
plt.show()

### first part (based on prev)

#### Eucledian distance

In [ ]:
overlap = 5
k=10
sim_mode="IoU"

In [ ]:
best_path = graph.best_path

In [ ]:
def simple_transform(path):
    return np.array([(ind // graph.y_len, ind % graph.y_len) for ind in path])

In [ ]:
from power_planner.utils.utils_ksp import KspUtils

In [ ]:
# set of all considered vertices
overall_set = set(best_path)
# helper set of only the best path
best_set = set(best_path)
# list of currently selected paths
best_paths = []
# same as above, but sets
best_path_sets = [simple_transform(best_path)]
# list of all that are considered
all_considered_paths = []
problematic_paths = []
for j, v_ind in enumerate(v_shortest):
    v = vertices[v_ind]
    if v not in overall_set:
        if int(graph.pred_map_ab[v]) == int(v) or int(graph.pred_map_ba[v]) == int(v):
            continue
        vertices_path = graph.compute_sp(v, source_v, target_v)
        sim_w_best = KspUtils.similarity(best_set, set(vertices_path), mode=sim_mode)
        if sim_w_best<overlap:
            overall_set.update(vertices_path)
            all_considered_paths.append(vertices_path)
        else:
            # do not care further if similarity with best path is already too big
            continue
        # check if diversity wrt to all others is fine
        trans_path = simple_transform(vertices_path)
        sofar = np.array([eucl_dist(trans_path, sp) for sp in best_path_sets])
        # print(sofar)
        # sofar = np.array([graphs.WeightedKSP.similarity(sp, set(vertices_path), sim_mode) for sp in best_path_sets])
        if np.all(sofar > overlap):
            # only store index of current path
            best_paths.append(len(all_considered_paths)-1)
            best_path_sets.append(trans_path)
            problematic_paths.append(0)
            print(j)
        else:
            a = np.where(sofar<=overlap)
            problematic_paths.append(a[-1][0])
    # stop if k paths have been found
    if len(best_paths) >= k:
        break

In [ ]:
best_paths

### consider preliminary result:

In [ ]:
len(all_considered_paths), len(v_shortest)

In [ ]:
ksp_prelim = [graph.transform_path(p) for p in np.array(all_considered_paths)[best_paths]]

In [ ]:
short_eval(ksp_prelim)

In [ ]:
def ksp_evaluate(ksp, sim_mode= "IoU"):
    costs = [k[2] for k in ksp]
    paths = [k[0] for k in ksp]
    
    path_sets = [set(p) for p in paths]
    
    if sim_mode=="total_nodes":
        return np.sum(costs), len(set.union(*path_sets))
    
    inters = []
    # iterate over all combinations
    for i in range(len(paths)):
        for j in range(i+1,len(paths)):
            if sim_mode == 'eucledian':
                p1 = paths[i]
                p2 = paths[j]
                norms = []
                for k in range(min([len(p1), len(p2)])):
                    norms.append(np.linalg.norm(np.array(p1[k])-np.array(p2[k])))
                inters.append(np.max(norms))
            else:
                print("hi")
                inters.append(similarity(path_sets[i], path_sets[j], mode=sim_mode))
    # print(inters)
    # assert(not any([i==1 for i in inters]))
    return np.sum(costs), np.sum(inters)

In [ ]:
ksp_evaluate(ksp_prelim, sim_mode="eucledian")

In [ ]:
plot_k_sp(ksp_prelim, graph.instance)

### plot all_considered

In [ ]:
best_paths

In [ ]:
arr = np.zeros(instance_corr.shape)
for path in all_considered_paths:
    path_trans = [(ind // graph.y_len, ind % graph.y_len) for ind in path]
    for (i,j) in path_trans:
        arr[i,j] = 1
        
for b in best_paths:
    path_trans = [(ind // graph.y_len, ind % graph.y_len) for ind in all_considered_paths[b]]
    for (i,j) in path_trans:
        arr[i,j]= 2

In [ ]:
plt.figure(figsize=(20,10))
plt.imshow(arr)
plt.colorbar()
plt.show()

## Refine

Now, we only need to lower the cost by good switching

In [ ]:
prob, prob_counts = np.unique(problematic_paths, return_counts=True)

In [ ]:
swap_path = best_paths[prob[np.argmax(prob_counts)]]

In [ ]:
swap_path

In [ ]:
min_path_len = np.min([len(p) for p in all_considered_paths])

In [ ]:
cut_paths = np.array([p[:min_path_len] for p in all_considered_paths])

In [ ]:
cut_paths.shape

## KSP evaluate

In [ ]:
def ksp_eval_eucledian(ksp):
    path_pos = [k[0] for k in ksp]
    # path_pos = [[(ind // graph.y_len, ind % graph.y_len) for ind in p] for p in paths]
    
    inters = []
    for i in range(len(path_pos)):
        for j in range(i+1,len(path_pos)):
            p1 = path_pos[i]
            p2 = path_pos[j]
            norms = []
            for k in range(min([len(p1), len(p2)])):
                norms.append(np.linalg.norm(np.array(p1[k])-np.array(p2[k])))
            inters.append(np.max(norms))
            print(i,j,inters[-1])
    for i,p in enumerate(path_pos):
        p = np.array(p)
        plt.plot(p[:,0], p[:,1], label=i)
    plt.legend()
    plt.show()

def compare_ksps(list_of_ksps, plot_labels, k, sim_mode="total_nodes"):
    for i, ksp_comp in enumerate(list_of_ksps):
        costs, sims = [], []
        for ksp in ksp_comp:
            if len(ksp)==k:
                cost, sim = ksp_evaluate(ksp, sim_mode=sim_mode)
                costs.append(cost)
                sims.append(sim)
        plt.plot(sims, costs, label=plot_labels[i])
    plt.title("comparison of ksp algorithms")
    plt.ylabel("sum of costs")
    plt.xlabel("similarity")
    plt.legend()
    plt.show()

### My new formulation: number of unique nodes in set

In [ ]:
def k_shortest_paths_new(self, source, dest, k, overlap=0.5):
    """
    Compute the k shortest path with minumum number of unique vertices
    Arguments:
        source: source vertex
        dest: target vertex
        k: number of paths to output
        overlap: maximum similarity
    """
    tic = time.time()
    THRESH = overlap * k * len(self.best_path)
    # sp_set = set(self.best_path)
    k_paths = [self.best_path]
    # save for each element in which paths it occurs
    occ_dict = defaultdict(list)
    for elem in self.best_path:
        occ_dict[elem].append(0)
    # occ_dict = {elem: [0] for elem in self.best_path}
    
    # get shortest paths
    vertices = np.unique(self.pos2node)[1:]
    v_dists = [self.dist_map_ab[v] + self.dist_map_ba[v] for v in vertices]
    v_shortest = np.argsort(v_dists)
    
    # iterate
    for j, v_ind in enumerate(v_shortest):
        v = vertices[v_ind]
        # v is already in one of the paths
        if v in occ_dict.keys():
            continue
        
        # get path itself
        if int(self.pred_map_ab[v]
                   ) == int(v) or int(self.pred_map_ba[v]) == int(v):
                continue
        # tic1 = time.time()
        try:
            path_ac = self.get_sp_from_preds(
                self.pred_map_ab, v, source
            )
            path_cb = self.get_sp_from_preds(self.pred_map_ba, v, dest)
        except RuntimeWarning:
            print("while loop not terminating")
            continue
        # times_getpath.append(time.time() - tic1)
        path_ac.reverse()
        # concatenate - leave 1 away because otherwise twice
        vertices_path = path_ac + path_cb[1:]

        # if we have less than k paths so far, add immediatly
        if len(k_paths)<k:
            k_paths.append(vertices_path)
            # sp_set.update(vertices_path)
            for v in vertices_path:
                occ_dict[v].append(len(k_paths)-1)
            continue
        # print(occ_dict)
        
        # implement path switch
        overlap_new = np.sum(np.array([u in occ_dict.keys() for u in vertices_path]))
        # skip first one because want to keep SP
        overlaps_prev = [0]
        for prev in k_paths[1:]:
            occs = np.array([len(occ_dict[v]) for v in prev])
            overlaps_prev.append(len(occs[occs>1]))
        
        # for k in range(len(k_paths)):
        #     print(k_paths[k], len(k_paths[k]))
        #     print(overlaps_prev[k])
        # evaluate(k_paths)
        
        # print(np.max(overlaps_prev), overlap_new)
        if np.max(overlaps_prev)> overlap_new:
            # TODO: flip --> if tie than last one
            # print([np.sum(p) for p in k_paths])
            dump_path_ind = np.argmax(overlaps_prev)
            del k_paths[dump_path_ind]
            # print("dumped", dump_path_ind)
            k_paths.insert(dump_path_ind, vertices_path)
            # remove indices from dict
            for key in occ_dict.keys():
                if dump_path_ind in occ_dict[key]:
                	occ_dict[key].remove(dump_path_ind)
            for key in vertices_path:
                occ_dict[key].append(dump_path_ind)
            # print("swapped", dump_path_ind)
        # print(occ_dict)
        check_cond = np.sum([1 for k in occ_dict.keys() if len(occ_dict[k])>0])
        if check_cond>THRESH:
            print("found end")
            print(check_cond, THRESH)
            break
    self.time_logs["ksp"] = round(time.time() - tic, 3)
    p_flat = list()
    for k in k_paths:
        p_flat.extend(k)
    return [self.transform_path(p) for p in k_paths]

In [ ]:
tic = time.time()
ksp = k_shortest_paths_new(graph, source_v, target_v, 10, overlap=0.5)
print(time.time()-tic)

### Compare formulations

In [ ]:
ksps_new = []
ksps_old = []
for i in [0.1, 0.3, 0.5, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9]:
    ksp = k_shortest_paths_new(graph, source_v, target_v, 10, overlap=i)
    ksps_new.append(ksp)
    ksp_old = graph.k_shortest_paths(source_v, target_v, 10, overlap=i)
    ksps_old.append(ksp_old)
iou_ksp = []
for i in [0.05, 0.07, 0.1, 0.3, 0.5, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.02]:
    ksp_old = graph.k_shortest_paths(source_v, target_v, 10, overlap=i, mode="eucledian")
    iou_ksp.append(ksp_old)

In [ ]:
compare_ksps([ksps_new, ksps_old, iou_ksp], ["new formulation", "set mode", "IoU"], 10, sim_mode="eucledian")

### Find out about dist_map and pred_map

In [ ]:
from graph_tool.all import Graph, shortest_distance
import numpy as np

In [ ]:
g = Graph()
g.add_vertex(20)
help = g.new_edge_property("float")
edges = []
for i in range(20):
    inds = np.random.permutation(20)[:3]
    for a in inds:
        edges.append([i+1,a+1,np.random.rand(1)])
g.add_edge_list(edges,eprops=[help])

In [ ]:
edges = [e for e in g.vertex(5).out_edges()]

In [ ]:
help[edges[2]]

In [ ]:
g.is_directed()

In [ ]:
dists_ab, preds_ab = shortest_distance(
            g,
            1,
            weights=help,
            directed=g.is_directed(),
            pred_map=True
        )
g.set_reversed(is_reversed=True)
dists_ba, preds_ba = shortest_distance(
            g,
            20,
            weights=help,
            directed=g.is_directed(),
            pred_map=True
        )

In [ ]:
dists_ab[5], dists_ba[5]